In [1]:
!pip install tweet-preprocessor
!pip install textblob
!pip install sastrawi
!pip install emoji
!pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 5.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=c62c4f6b765692695d0c9418654ea9fa40b9feba253f9937794e56a6730a8d49
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 5.2 MB/s 


In [2]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
from preprocessor.api import clean, tokenize, parse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import datetime
from datetime import timedelta
import numpy as np
import emoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [3]:
#Read data
import pandas as pd

def load_data():
  data = pd.read_csv('/content/mudikLebaran_fix.csv')
  return data

In [4]:
tweet_df = load_data()

#Buka data tweet yang berisi Waktu, ID, User, Tweet
tweet_df = pd.DataFrame(tweet_df[['Waktu', 'ID', 'User', 'Tweet']])
tweet_df.head(5)

,Waktu,ID,User,Tweet
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ..."
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...


**Preprocessing**

In [5]:
#Cleansing

#remove mention tweet dan retweet
def remove_pattern(text, pattern_regex):
  r = re.findall(pattern_regex, text)
  for i in r:
    text = re.sub(i, ' ', text)
  return text

In [6]:
tweet_df['clean_tweet'] = np.vectorize(remove_pattern)(tweet_df['Tweet'], " *RT* | *@[\w]")
tweet_df.head(5)

,Waktu,ID,User,Tweet,clean_tweet
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ..."
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...,akimiNOW: Lebaran tahun ini istimewa karena ...
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...


In [7]:
#remove simbol 
def remove(text) :
  text = ' '.join(re.sub("(@[A-Za-z0-9]+) | ([^0-9A-Za-z \t]) | (\w+:\/\/\S+)", " ", text).split())
  return text
tweet_df['remove_http'] = tweet_df['clean_tweet'].apply(lambda x:remove(x))
tweet_df.head()

,Waktu,ID,User,Tweet,clean_tweet,remove_http
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ..."
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...,akimiNOW: Lebaran tahun ini istimewa karena ...,akimiNOW: Lebaran tahun ini istimewa karena ki...
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...


In [8]:
#remove
def remove(tweet):
  #remove stock market tickers like $GE
  tweet = re.sub(r'\$\w*', ' ', tweet)

  #remove old style retweet text "RT"
  tweet = re.sub(r'^RT[\s]+', ' ', tweet)

  #remove hastags
  #only removing the #sign from the word 
  tweet = re.sub(r'#', ' ', tweet)
  #remove angka
  tweet = re.sub('[0-9]+', ' ', tweet)

  return tweet

tweet_df['remove_hastag'] = tweet_df['remove_http'].apply(lambda x:remove(x))
tweet_df.head()

,Waktu,ID,User,Tweet,clean_tweet,remove_http,remove_hastag
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ..."
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...,akimiNOW: Lebaran tahun ini istimewa karena ...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akimiNOW: Lebaran tahun ini istimewa karena ki...
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran Dini...


In [9]:
#remove data duplikat
tweet_df.drop_duplicates(subset = "remove_hastag", keep = 'first', inplace = True)
tweet_df.head()

,Waktu,ID,User,Tweet,clean_tweet,remove_http,remove_hastag
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ..."
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...,akimiNOW: Lebaran tahun ini istimewa karena ...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akimiNOW: Lebaran tahun ini istimewa karena ki...
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran Dini...


In [10]:
#Case Folding (Tokenizing, Filtering, Stemming)

#import stopword
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

stop_factory = StopWordRemoverFactory().get_stop_words()
more_stopwords = [
                  'yg', 'utk', 'cuman', 'deh', 'btw', 'tapi', 'gua', 'gue', 'lo', 'lu',
                  'kalo', 'trs', 'jd', 'nih', 'ntar', 'nya', 'lg', 'gk', 'dpt', 'ecusli',
                  'sih', 'eh', 'bang', 'br', 'kyk', 'rp', 'jt', 'hahaha', 'yang', 'weh', 'tuh', 
                  'wkwkwk', 'wkwk', 'sm', 'sdh', 'guys', 'thx', 'pls', 'kzl', 'si', 'iy', 'btw',
                  'gpp', 'oi', 'gw', 'jg'
]
data = stop_factory + more_stopwords

dictionary = ArrayDictionary(data)
str = StopWordRemover(dictionary)

print(data)

#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize 
from nltk.tokenize import TweetTokenizer

#Happy Emoticons
emoticons_happy = set([
                 ':)', ':-)', ';)', ':0)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
                 ':^)', ':-D', '8-D', '8D', 'x-D', 'X-D', 'XD', '=-D', '=D',
                 '=-3', '=3', ':-))', ":-')", ":')", ':*', ':*^', '>:p', ':-p', ':p', 'X-p',
                 'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)', '<3' 
])

emoticon_sad = set([
                    ':(', ":'(", ':<', ':@', ':{', ':c', ':-<', ';-[', ':\\', ':S', ':-/'
])

#all emoticons (happy+sad)
emoticons = emoticons_happy.union(emoticon_sad)

def clean_tweets(tweet):
  #tokenize tweets 
  tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
  tweet_tokens = tokenizer.tokenize(tweet)

  tweets_clean = []
  for word in tweet_tokens:
    if (word not in data and #remove stopword
        word not in emoticons and #remove emoticons
        word not in string.punctuation):
            #tweet_clean.append(word)
            stem_word = stemmer.stem(word) #stemming word
            tweets_clean.append(stem_word)
  return tweets_clean

tweet_df['Tweet_bersih'] = tweet_df['remove_hastag'].apply(lambda x: clean_tweets(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['a', 'ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'arti', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'b', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagainamakah', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasannya', 'bahwasanya', 'baik', 'baiklah', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'ber

In [11]:
#hasil tokenisasi
tweet_df.head()

,Waktu,ID,User,Tweet,clean_tweet,remove_http,remove_hastag,Tweet_bersih
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,"[libur, lebaran, mudik, lebaran, kendara, umum..."
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,"[lebaran, taun, kmren, tiga, sepi, pol, rasany..."
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","[tahun, mudik, lebaran, boleh, mudik, habis, t..."
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...,akimiNOW: Lebaran tahun ini istimewa karena ...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akimiNOW: Lebaran tahun ini istimewa karena ki...,"[akiminow, lebaran, tahun, istimewa, mudik, pu..."
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran Dini...,"[rekayasa, lintas, arus, mudik, lebaran, nilai..."


In [12]:
#remove punct
def remove_punct(text):
  text = " ".join([char for char in text if char not in string.punctuation])
  return text

tweet_df['Tweet_bersih'] = tweet_df['Tweet_bersih'].apply(lambda x: remove_punct(x))

In [13]:
#hasil punctuation
tweet_df.head()

,Waktu,ID,User,Tweet,clean_tweet,remove_http,remove_hastag,Tweet_bersih
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,libur lebaran mudik lebaran kendara umum tenan...
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,lebaran taun kmren tiga sepi pol rasanyaa repo...
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...",tahun mudik lebaran boleh mudik habis tiket al...
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...,akimiNOW: Lebaran tahun ini istimewa karena ...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akiminow lebaran tahun istimewa mudik pulang k...
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran Dini...,rekayasa lintas arus mudik lebaran nilai sukes...


In [14]:
#remove tweet kosong 
tweet_df = tweet_df[tweet_df['Tweet_bersih']!=' ']

In [15]:
#hasil remove tweet kosong
tweet_df.head()

,Waktu,ID,User,Tweet,clean_tweet,remove_http,remove_hastag,Tweet_bersih
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,libur lebaran mudik lebaran kendara umum tenan...
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,lebaran taun kmren tiga sepi pol rasanyaa repo...
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...",tahun mudik lebaran boleh mudik habis tiket al...
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...,akimiNOW: Lebaran tahun ini istimewa karena ...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akiminow lebaran tahun istimewa mudik pulang k...
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran Dini...,rekayasa lintas arus mudik lebaran nilai sukes...


In [16]:
#reset index
tweet_df = tweet_df.reset_index(drop=True)
tweet_df.head()

,Waktu,ID,User,Tweet,clean_tweet,remove_http,remove_hastag,Tweet_bersih
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,libur lebaran mudik lebaran kendara umum tenan...
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,lebaran taun kmren tiga sepi pol rasanyaa repo...
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...",tahun mudik lebaran boleh mudik habis tiket al...
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...,akimiNOW: Lebaran tahun ini istimewa karena ...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akiminow lebaran tahun istimewa mudik pulang k...
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran Dini...,rekayasa lintas arus mudik lebaran nilai sukes...


In [17]:
tweet_df.drop_duplicates(subset = "remove_hastag", keep = 'first', inplace = True)
tweet_df.head()

,Waktu,ID,User,Tweet,clean_tweet,remove_http,remove_hastag,Tweet_bersih
0,2022-05-01 23:59:00,1520915722704105472,newsmerahputih,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,libur lebaran mudik lebaran kendara umum tenan...
1,2022-05-01 23:58:51,1520915688181104641,diana sari,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,Kalo lebaran taun kmren cuman bertiga sepi pol...,lebaran taun kmren tiga sepi pol rasanyaa repo...
2,2022-05-01 23:58:51,1520915687115476994,detikinet,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...","Tahun ini, mudik Lebaran mulai diperbolehkan. ...",tahun mudik lebaran boleh mudik habis tiket al...
3,2022-05-01 23:58:40,1520915639543934976,DPW PKB JAWA TIMUR,RT @cakimiNOW: Lebaran tahun ini istimewa kare...,akimiNOW: Lebaran tahun ini istimewa karena ...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akimiNOW: Lebaran tahun ini istimewa karena ki...,akiminow lebaran tahun istimewa mudik pulang k...
4,2022-05-01 23:58:34,1520915616785338369,TEMPO.CO,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran 2022 D...,Rekayasa Lalu Lintas Arus Mudik Lebaran Dini...,rekayasa lintas arus mudik lebaran nilai sukes...


In [18]:
#remove kolom
tweet_df.drop(tweet_df.columns[[0,1,2,3,4,5]], axis = 1, inplace= True)
tweet_df.head()

,remove_hastag,Tweet_bersih
0,Libur Lebaran sudah tiba. Kamu mudik lebaran d...,libur lebaran mudik lebaran kendara umum tenan...
1,Kalo lebaran taun kmren cuman bertiga sepi pol...,lebaran taun kmren tiga sepi pol rasanyaa repo...
2,"Tahun ini, mudik Lebaran mulai diperbolehkan. ...",tahun mudik lebaran boleh mudik habis tiket al...
3,akimiNOW: Lebaran tahun ini istimewa karena ki...,akiminow lebaran tahun istimewa mudik pulang k...
4,Rekayasa Lalu Lintas Arus Mudik Lebaran Dini...,rekayasa lintas arus mudik lebaran nilai sukes...


In [19]:
tweet_df.drop(tweet_df.columns[[0]], axis = 1, inplace= True)
tweet_df.head()

,Tweet_bersih
0,libur lebaran mudik lebaran kendara umum tenan...
1,lebaran taun kmren tiga sepi pol rasanyaa repo...
2,tahun mudik lebaran boleh mudik habis tiket al...
3,akiminow lebaran tahun istimewa mudik pulang k...
4,rekayasa lintas arus mudik lebaran nilai sukes...


In [20]:
#simpen data bersih 
tweet_df.to_csv('mudikLebaran_dataBersih_new.csv', encoding ='utf-8', index=False)

In [21]:
!pip3 install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 1.3 MB 11.0 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=1235bb4b96085344678dab371eeb5c087442554fc5767c102487f010706c4622
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [22]:
#importing library
import pandas as pd
import googletrans
from googletrans import Translator

In [24]:
#Membaca dan menyimpan file csv sebagai dataframe
df = pd.read_csv('mudikLebaran_dataBersih_new.csv')
tweet_df.head(5)

,Tweet_bersih
0,libur lebaran mudik lebaran kendara umum tenan...
1,lebaran taun kmren tiga sepi pol rasanyaa repo...
2,tahun mudik lebaran boleh mudik habis tiket al...
3,akiminow lebaran tahun istimewa mudik pulang k...
4,rekayasa lintas arus mudik lebaran nilai sukes...


**Translate**

In [25]:
translator = Translator()
translations = {}
for column in df.columns:
  #unique element of the coloumn
  unique_element = df[column].unique()
  for element in unique_element:
    #Mentranslate
    translations[element] = translator.translate(element).text
translations

{'abiylarisfa lumayan iri orang ngepost long holiday kangen mudik suasana siap xe xa': 'abiylarisfa is quite jealous of people posting long holiday miss going home, the atmosphere is ready xe xa',
 'akiminow lebaran tahun istimewa mudik pulang kampung jati titik nol restart xe xa': 'akiminow Eid special year going home to teak, zero point restart xe xa',
 'aktivitas lebaran gak mudik shalat ied an dgn tetangga an dgn keluarga rmh xe xa': "Eid activities don't go home and pray Eid with neighbors and family at home xe xa",
 'aleyluhooo kepala negara syukur momen lebaran tahun masyarakat boleh mudik berkum xe xa': 'haleyluhoo, head of state, thanksgiving for the moment of Eid in the year, the public can go home together xe xa',
 'alhamdulilah lebaran tahun mudik blitar nyenengin mamah plg kampung nyaa xf xa xf xa': 'Alhamdulilah, Lebaran, the year of going home in Blitar, makes mommy happy, please go back to her village xf xa xf xa',
 'an mar tol penuh jelang lebaran ukur ekonomi tumbuh n

In [26]:
#Mengganti semua kata yang diterjemahkan dari kamus ke original dataframe
df.replace(translations, inplace = True)
df.head(10)

,Tweet_bersih
0,Eid holiday homecoming Lebaran public transpor...
1,"Last year's Eid, three were quiet, Pol. It fel..."
2,"Eid homecoming year, you can go home and you c..."
3,"akiminow Eid special year going home to teak, ..."
4,"cross-flow engineering for Lebaran, the value ..."
5,Nasdem faction Eid al-Fitr Happy Eid Al-Fitr s...
6,"Ministry of Defence, Rev. President, Joko Wido..."
7,"Minalaidzin Walfaidzin, sorry to be born and i..."
8,"chiiirin happy Eid, mas Bayu, Win, I love goin..."
9,I'm so happy to see Lebaran instastory before ...


In [27]:
#menympan data hasil translate
df.to_csv('translate_mudikLebaran_new.csv', encoding='utf-8', index=False)